In [1]:

from generate_dataset import Dataset

dataset = Dataset.from_json("dataset_top50.json")

boards = list(dataset.boards.values())
boards



[Board(state=[1, 2, 3, 4, 5, 'X', 'O', 8, 9], player='X', scores=[Score(value=0, depth=9, move=1), Score(value=-1, depth=8, move=2), Score(value=-1, depth=8, move=3), Score(value=-1, depth=8, move=4), Score(value=0, depth=9, move=5), Score(value=-1, depth=8, move=8), Score(value=1, depth=7, move=9)], category=Category(choice_complexity=(6, 7), depth_complexity=5, decision='offensive', pattern='threat-fork', symmetry_group='12345XO89|123X5678O|12O4567X9|12OX56789|1X345678O|1X3456O89|O234567X9|O2345X789')),
 Board(state=[1, 2, 3, 4, 5, 'X', 'O', 'X', 9], player='O', scores=[Score(value=1, depth=8, move=1), Score(value=0, depth=9, move=2), Score(value=-1, depth=7, move=3), Score(value=-1, depth=9, move=4), Score(value=0, depth=9, move=5), Score(value=-1, depth=7, move=9)], category=Category(choice_complexity=(5, 6), depth_complexity=5, decision='offensive', pattern='threat-fork', symmetry_group='12345XOX9|123X567XO|12O45X7X9|1X345X78O|1X3X56O89|1XOX56789|O23X567X9|OX345X789')),
 Board(sta

In [31]:
from generate_dataset import Board
import random

def plot_board(state):
    board = ['-' if cell not in ['X', 'O'] else cell for cell in state]
    return f" {board[0]} | {board[1]} | {board[2]} \n---+---+---\n {board[3]} | {board[4]} | {board[5]} \n---+---+---\n {board[6]} | {board[7]} | {board[8]} "

# Shuffle the boards
shuffled_boards = boards.copy()
random.shuffle(shuffled_boards)

with open('tictactoe_human_benchmark.txt', 'w') as f:
    f.write("┌─────────────────────────────────────────────┐\n")
    f.write("│ Welcome to the Tic-Tac-Toe Human Benchmark! │\n")
    f.write("└─────────────────────────────────────────────┘\n\n")

    f.write("INSTRUCTIONS:\n")
    f.write("The 'X' player always goes first. You have to infer your mark from the board.\n\n")
    f.write("The available moves in a board are numbered as follows:\n\n")
    f.write(" 1 | 2 | 3 \n---+---+---\n 4 | 5 | 6 \n---+---+---\n 7 | 8 | 9 \n\n")
    f.write("To submit a move, fill in the argument in choice(position). For example, choice(5) selects the center position.\n")
    f.write("These are the only edits you should make to this file.\n")
    f.write("I omitted the numbering for better readability in the actual test.\n")
    f.write("Try to spend at MOST 10 to 20 seconds on each board. As if you were casually playing with another person in real life.\n")
    f.write("Below there are 50 boards of Tic-Tac-Toe.\n\n\n\n")
        
    f.write("─────────────────────────TEST BEGINS─────────────────────────\n\n")

    for i, board in enumerate(shuffled_boards, 1):
        board_hash = Board.hash_board(board.state)
        f.write(f"# {i}\n")
        f.write(f"{board_hash} -> choice()\n")
        f.write(plot_board(board.state))
        f.write("\n\n")

print("Tictactoe boards have been written to 'tictactoe_human_benchmark.txt'")


Tictactoe boards have been written to 'tictactoe_human_benchmark.txt'


# Parse Results

In [1]:
import os
import re
import pandas as pd

def parse_participant_responses():
    folder_path = 'human_performance/complete_questionnaires'
    participant_responses = {}
    
    # Regex pattern to match board_hash and choice
    pattern = r'(\w{9})\s*->\s*choice\((\d)\)'
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            participant_id = filename.split('_')[-1].split('.')[0]
            participant_responses[participant_id] = []
            
            with open(os.path.join(folder_path, filename), 'r') as f:
                content = f.read()
                matches = re.findall(pattern, content)
                for board_hash, choice in matches:
                    participant_responses[participant_id].append((board_hash, int(choice)))
    
    return participant_responses

participant_responses = parse_participant_responses()

# Create a DataFrame
df = pd.DataFrame(columns=['board_hash'] + list(participant_responses.keys()))

# Fill the DataFrame
for participant, responses in participant_responses.items():
    for board_hash, choice in responses:
        if board_hash not in df['board_hash'].values:
            new_row = pd.DataFrame({'board_hash': [board_hash]})
            df = pd.concat([df, new_row], ignore_index=True)
        df.loc[df['board_hash'] == board_hash, participant] = choice

# Check if parsing was successful
if len(df) == 50 and all(df[participant].count() == 50 for participant in participant_responses.keys()):
    print("Parsing successful: 50 entries for each participant with matching board hashes.")
else:
    print("Parsing error: Inconsistent number of entries or mismatched board hashes.")

df

Parsing successful: 50 entries for each participant with matching board hashes.


,board_hash,B,A,C
0,12XX5OOOX,1,1,1
1,123O5X7X9,1,3,9
2,1O3OXX7X9,1,1,1
3,123456OXX,5,5,3
4,123O5XX89,9,9,9
5,123456XOX,5,5,5
6,12345XO89,5,5,3
7,12O45XXXO,1,1,1
8,12345XX8O,5,5,1
9,12OX5O78X,7,1,1


In [3]:
from generate_dataset import Dataset, Board

dataset = Dataset.from_json("dataset_top50.json")

boards = []
for board in dataset.boards.values():
    state = Board.hash_board(board.state)  # Convert state to hash
    scores = board.scores
    highest_score = max(score.value for score in scores)
    highest_moves = [score.move for score in scores if score.value == highest_score]
    boards.append({
        'state': state,
        'moves': highest_moves
    })

boards

[{'state': '12345XO89', 'moves': [9]},
 {'state': '12345XOX9', 'moves': [1]},
 {'state': '12OX5678X', 'moves': [1]},
 {'state': '12O45XXO9', 'moves': [4]},
 {'state': '123OOXX89', 'moves': [9]},
 {'state': '123O5XXO9', 'moves': [3]},
 {'state': '123OOX7X9', 'moves': [9]},
 {'state': '1O3O5X7X9', 'moves': [9]},
 {'state': '12O456XOX', 'moves': [1]},
 {'state': '12O45XX8O', 'moves': [4]},
 {'state': '1OXX5O7X9', 'moves': [7]},
 {'state': '12XO5X7XO', 'moves': [1]},
 {'state': '1O3OXX7X9', 'moves': [1]},
 {'state': '12O45XXXO', 'moves': [1]},
 {'state': '12O45XXOX', 'moves': [2]},
 {'state': '12O456X89', 'moves': [1, 9]},
 {'state': '123O5X7X9', 'moves': [3, 9]},
 {'state': '123456OXX', 'moves': [1, 4]},
 {'state': '12OX567X9', 'moves': [1, 9]},
 {'state': '12O4X6X89', 'moves': [1, 9]},
 {'state': '123O5XX89', 'moves': [3, 9]},
 {'state': '1OXX5O7XO', 'moves': [7]},
 {'state': '12XX5OOXO', 'moves': [2]},
 {'state': '12XX5OOOX', 'moves': [1]},
 {'state': '1OXXOO7X9', 'moves': [7]},
 {'stat

In [5]:
# Initialize new columns for scores
for participant in ['A', 'B', 'C']:
    df[f'{participant}_score'] = 0

# Convert boards list to a dictionary for faster lookup
boards_dict = {board['state']: board['moves'] for board in boards}

# Score each participant's choice
for index, row in df.iterrows():
    board_hash = row['board_hash']
    if board_hash in boards_dict:
        best_moves = boards_dict[board_hash]
        for participant in ['A', 'B', 'C']:
            if row[participant] in best_moves:
                df.at[index, f'{participant}_score'] = 1

# Calculate and print total scores and average score
total_scores = {}
for participant in ['A', 'B', 'C']:
    total_score = df[f'{participant}_score'].sum()
    total_scores[participant] = total_score
    print(f"Total score for {participant}: {total_score}/50")

avg_score = sum(total_scores.values()) / len(total_scores)
print(f"Average score across all participants: {avg_score:.2f}/50")

df


Total score for A: 43/50
Total score for B: 41/50
Total score for C: 42/50
Average score across all participants: 42.00/50


,board_hash,B,A,C,A_score,B_score,C_score
0,12XX5OOOX,1,1,1,1,1,1
1,123O5X7X9,1,3,9,1,0,1
2,1O3OXX7X9,1,1,1,1,1,1
3,123456OXX,5,5,3,0,0,0
4,123O5XX89,9,9,9,1,1,1
5,123456XOX,5,5,5,1,1,1
6,12345XO89,5,5,3,0,0,0
7,12O45XXXO,1,1,1,1,1,1
8,12345XX8O,5,5,1,1,1,0
9,12OX5O78X,7,1,1,1,1,1


In [6]:
import json
import os

# Convert DataFrame to a list of dictionaries
data = df.to_dict(orient='records')

# Convert to JSON
json_data = json.dumps(data, indent=2)

# Ensure the human_tests folder exists
os.makedirs('human_tests', exist_ok=True)

# Save to a file in the human_tests folder
file_path = os.path.join('human_tests', 'participant_scores.json')
with open(file_path, 'w') as f:
    f.write(json_data)

print(f"Data has been saved to '{file_path}'")


Data has been saved to 'human_tests/participant_scores.json'
